<a href="https://colab.research.google.com/github/Aljesh-basnet/gif_auto_audio_description/blob/Dev/Aljesh_3DCNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import imageio
import numpy as np

In [18]:

import pandas as pd
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Step 2: Load TSV file into a DataFrame without using the first line as the header
file_path = '/content/drive/My Drive/tgif-v1.0.tsv'  # Correct file path
df = pd.read_csv(file_path, sep='\t', header=None)  # Load TSV

# Display the first few rows of the DataFrame
df.head()


Mounted at /content/drive


,0,1
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ..."
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...


In [19]:

df.columns = ['Image', 'Text']
df.head()

,Image,Text
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ..."
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...


In [20]:
num_frames = 5

In [21]:
 # Function to extract frames from GIF
# Assuming your GIF files are in a directory named 'gifs'
import imageio
import os
def extract_frames(gif_path, num_frames=3):
    try:
        gif = imageio.mimread(gif_path)
        total_frames = len(gif)
        if total_frames == 0:
            raise ValueError("No frames found in GIF.")
        interval = max(total_frames // num_frames, 1)
        selected_frames = [gif[i] for i in range(0, total_frames, interval)][:num_frames]
        return selected_frames
    except Exception as e:
        print(f"Error extracting frames from {gif_path}: {e}")
        return []

# Example Usage (replace with your actual GIF file paths)



In [22]:

# Function to extract GIF and save frames as image links
def extract_and_save_frames(gif_path, num_frames=num_frames):
    try:
        frames = extract_frames(gif_path, num_frames)
        image_links = []
        for i, frame in enumerate(frames):
            image_filename = f"{os.path.basename(gif_path)}_frame_{i}.png"
            imageio.imwrite(image_filename, frame)  # Save the frame as a PNG file
            image_links.append(image_filename)  # Store the image filename (or URL)
        return image_links
    except Exception as e:
        print(f"Error extracting and saving frames from {gif_path}: {e}")
        return []



In [23]:
# Create a new column in the DataFrame to store image links
n_rows=1000
new_df= df.iloc[:n_rows].copy()
new_df['Image_Links'] = new_df['Image'].apply(extract_and_save_frames)


In [24]:
new_df.head()

,Image,Text,Image_Links
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ...",[tumblr_mevmyaKtDf1rgvhr8o1_500.gif_frame_0.pn...
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet,[tumblr_nok4eeONTv1s2yegdo1_400.gif_frame_0.pn...
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.,[tumblr_mllh01J96X1s9npefo1_250.gif_frame_0.pn...
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle,[tumblr_nqlr0rn8ox1r2r0koo1_400.gif_frame_0.pn...
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...,[tumblr_mvtuwlhSkE1qbnleeo1_500.gif_frame_0.pn...


In [25]:
X = new_df['Image_Links']
y = new_df['Text']

In [26]:
X.shape

(1000,)

In [27]:
y.shape

(1000,)

In [28]:
X.head()

,Image_Links
0,[tumblr_mevmyaKtDf1rgvhr8o1_500.gif_frame_0.pn...
1,[tumblr_nok4eeONTv1s2yegdo1_400.gif_frame_0.pn...
2,[tumblr_mllh01J96X1s9npefo1_250.gif_frame_0.pn...
3,[tumblr_nqlr0rn8ox1r2r0koo1_400.gif_frame_0.pn...
4,[tumblr_mvtuwlhSkE1qbnleeo1_500.gif_frame_0.pn...


In [29]:
y.head()

,Text
0,"a man is glaring, and someone with sunglasses ..."
1,a cat tries to catch a mouse on a tablet
2,a man dressed in red is dancing.
3,an animal comes close to another in the jungle
4,a man in a hat adjusts his tie and makes a wei...


In [30]:
# prompt: video to array for x

import numpy as np
import cv2

def video_to_array(image_links, frame_size=(128, 128), max_frames=num_frames):
    """
    Converts a list of image links into a NumPy array of frames, suitable for 3D CNN input.

    Args:
    - image_links (list): A list of image file paths.
    - frame_size (tuple): Desired size of each frame (width, height).
    - max_frames (int): Maximum number of frames to extract from the video.

    Returns:
    - video_array (np.ndarray): A 4D NumPy array of shape (num_frames, height, width, channels).
    """
    frames = []
    for link in image_links:
        try:
            frame = cv2.imread(link)
            if frame is not None:
                # Resize the frame to the desired size (frame_size[1], frame_size[0])
                frame_resized = cv2.resize(frame, frame_size)

                # Convert BGR (OpenCV default) to RGB
                frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

                # Append the frame to the list
                frames.append(frame_rgb)

            if len(frames) >= max_frames:
                break

        except Exception as e:
            print(f"Error processing image {link}: {e}")

    # Pad frames if there are fewer than max_frames
    while len(frames) < max_frames:
        frames.append(np.zeros((frame_size[1], frame_size[0], 3), dtype=np.uint8))


    # Convert the list of frames to a NumPy array
    video_array = np.array(frames)

    # Normalize pixel values to range [0, 1]
    video_array = video_array.astype('float32') / 255.0

    return video_array

# Example usage:
# Assuming 'X' is a column with image links in your DataFrame

X_array = X.apply(video_to_array)


print(X_array.shape)
X_array = np.array([arr for arr in X_array])

print(X_array.shape)


(1000,)
(1000, 5, 128, 128, 3)


In [31]:
# prompt: normalization of x_array

import numpy as np

# Assuming X_array is your 5D numpy array
# Reshape X_array to a 2D array
X_array_reshaped = X_array.reshape(X_array.shape[0], -1)

# Calculate the mean and standard deviation
mean = np.mean(X_array_reshaped, axis=0)
std = np.std(X_array_reshaped, axis=0)

# Normalize the data
X_array_normalized = (X_array_reshaped - mean) / std

# Reshape back to the original shape
X_array_normalized = X_array_normalized.reshape(X_array.shape)

print(X_array_normalized.shape)


(1000, 5, 128, 128, 3)


In [33]:
# prompt: preprocess y and remove stop words

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
  """
  Preprocesses text by lowercasing, tokenizing, removing stop words, and joining tokens.
  """
  text = text.lower()
  tokens = word_tokenize(text)
  stop_words = set(stopwords.words('english'))
  filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
  processed_text = " ".join(filtered_tokens)
  return processed_text


y_processed = y.apply(preprocess_text)
print(y_processed.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


0    man glaring someone sunglasses appears
1              cat tries catch mouse tablet
2                   man dressed red dancing
3         animal comes close another jungle
4      man hat adjusts tie makes weird face
Name: Text, dtype: object


In [34]:
# prompt: preprocess y

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming 'new_df' contains your DataFrame with text captions
y_captions = new_df['Text'].tolist()

# Tokenize the captions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(y_captions)
y_sequences = tokenizer.texts_to_sequences(y_captions)

# Pad sequences to a fixed length
max_caption_length = 20  # You might need to adjust this based on your data
y_data = pad_sequences(y_sequences, maxlen=max_caption_length, padding='post')

# Convert to categorical if you're using a categorical cross-entropy loss
# num_words = len(tokenizer.word_index) + 1
# y_data = to_categorical(y_data, num_classes=num_words)

print("Shape of y_data:", y_data.shape)


Shape of y_data: (1000, 20)


In [35]:
# prompt: check tokens of words

print(tokenizer.word_index)


{'a': 1, 'is': 2, 'and': 3, 'man': 4, 'the': 5, 'in': 6, 'woman': 7, 'are': 8, 'with': 9, 'on': 10, 'his': 11, 'of': 12, 'to': 13, 'her': 14, 'girl': 15, 'two': 16, 'young': 17, 'while': 18, 'at': 19, 'hair': 20, 'dancing': 21, 'wearing': 22, 'black': 23, 'boy': 24, 'down': 25, 'up': 26, 'head': 27, 'white': 28, 'looking': 29, 'other': 30, 'one': 31, 'it': 32, 'something': 33, 'an': 34, 'men': 35, 'talking': 36, 'out': 37, 'smiling': 38, 'holding': 39, 'person': 40, 'each': 41, 'walking': 42, 'as': 43, 'around': 44, 'hand': 45, 'cat': 46, 'then': 47, 'into': 48, 'guy': 49, 'another': 50, 'singing': 51, 'from': 52, 'shirt': 53, 'kissing': 54, 'hands': 55, 'face': 56, 'he': 57, 'someone': 58, 'looks': 59, 'people': 60, 'red': 61, 'doing': 62, 'some': 63, 'couple': 64, 'through': 65, 'by': 66, 'bed': 67, 'over': 68, 'moving': 69, 'front': 70, 'away': 71, 'sitting': 72, 'dark': 73, 'hat': 74, 'laughing': 75, 'off': 76, 'stage': 77, 'car': 78, 'back': 79, 'him': 80, 'microphone': 81, 'has':

In [36]:
X_array

array([[[[[0.58431375, 0.5568628 , 0.5058824 ],
          [0.5921569 , 0.5686275 , 0.5176471 ],
          [0.5921569 , 0.5647059 , 0.5137255 ],
          ...,
          [0.44313726, 0.42745098, 0.4       ],
          [0.6745098 , 0.65882355, 0.627451  ],
          [0.8666667 , 0.8509804 , 0.8235294 ]],

         [[0.5921569 , 0.5647059 , 0.5137255 ],
          [0.59607846, 0.5686275 , 0.5176471 ],
          [0.5882353 , 0.56078434, 0.5137255 ],
          ...,
          [0.52156866, 0.5058824 , 0.48235294],
          [0.5647059 , 0.54901963, 0.5176471 ],
          [0.5764706 , 0.56078434, 0.52156866]],

         [[0.58431375, 0.56078434, 0.5137255 ],
          [0.57254905, 0.54509807, 0.5019608 ],
          [0.5764706 , 0.54901963, 0.49411765],
          ...,
          [0.34117648, 0.3254902 , 0.30588236],
          [0.4392157 , 0.41960785, 0.4       ],
          [0.56078434, 0.54509807, 0.5176471 ]],

         ...,

         [[0.        , 0.        , 0.        ],
          [0.        ,

In [37]:
num_words = len(tokenizer.word_index) + 1  # Calculate the number of unique words
y_data_categorical = to_categorical(y_data, num_classes=num_words) # Use num_words as num_classes
y_data_categorical.shape

(1000, 20, 1682)

In [38]:
# Example usage
input_shape = (num_frames, 128, 128, 3)  # (num_frames, height, width, channels)
num_classes = num_words  # Number of classes for classification
num_classes

1682

In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape


def create_3dcnnlstm(input_shape, num_classes):
    """
    Creates a 3D CNN-LSTM model for spatiotemporal tasks (e.g., video classification).

    Parameters:
    - input_shape: Tuple of the shape (num_frames, height, width, channels), e.g., (16, 128, 128, 3)
    - num_classes: The number of output classes for classification.

    Returns:
    - model: A compiled Keras model.
    """

    model = Sequential()


    # 3D CNN layers for spatial and temporal feature extraction
    model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))  # Pool only over height and width, not depth

    model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu', padding='same'))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same'))

    # Flatten the 3D CNN output and reshape for the LSTM
    model.add(Flatten())
    model.add(Reshape((20, -1))) # Reshape to (timesteps, features) for LSTM

    # LSTM layers for temporal modeling
    model.add(LSTM(128, return_sequences=True))  # You can set return_sequences=True if stacking LSTMs

    # Dropout for regularization
    model.add(Dropout(0.5))

    # Fully connected layer
    model.add(TimeDistributed(Dense(128, activation='relu')))

    # Output layer with softmax for multi-class classification
    model.add(TimeDistributed(Dense(num_classes, activation='softmax')))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model



model = create_3dcnnlstm(input_shape, num_classes)
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 5, 128, 128, 32)     │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d (MaxPooling3D)         │ (None, 5, 64, 64, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 5, 64, 64, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_1 (MaxPooling3D)       │ (None, 5, 32, 32, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_2 (Conv3D)                    │ (None, 5, 32, 32, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_2 (MaxPooling3D)       │ (None, 5, 16, 16, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 163840)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 20, 8192)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 20, 128)             │       4,260,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 20, 128)             │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 20, 1682)            │         216,978 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,773,138 (18.21 MB)

 Trainable params: 4,773,138 (18.21 MB)

 Non-trainable params: 0 (0.00 B)

In [40]:
# prompt: train test split for x_array_normalized and y_data_categorical

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_array_normalized, y_data_categorical, test_size=0.2, random_state=42
)


In [41]:
# Fit the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 17s 308ms/step - accuracy: 0.3567 - loss: 6.0953 - val_accuracy: 0.4625 - val_loss: 3.8457
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 12s 254ms/step - accuracy: 0.4567 - loss: 3.7900 - val_accuracy: 0.4625 - val_loss: 3.6888
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 243ms/step - accuracy: 0.4479 - loss: 3.7301 - val_accuracy: 0.4625 - val_loss: 3.6548
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 242ms/step - accuracy: 0.4403 - loss: 3.6975 - val_accuracy: 0.4625 - val_loss: 3.6244
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 251ms/step - accuracy: 0.4478 - loss: 3.6313 - val_accuracy: 0.4625 - val_loss: 3.5871
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 245ms/step - accuracy: 0.4363 - loss: 3.6393 - val_accuracy: 0.4625 - val_loss: 3.5130
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 244ms/step - accuracy: 0.4438 - loss: 3.5644 - val_accuracy: 0.4625 - val_loss: 3.4731
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 258ms/step - accuracy: 0.4579 - loss: 3.4715 - val_accuracy: 

In [42]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape, Input, Bidirectional


def create_3dcnnlstm(input_shape, num_classes):
    """
    Creates a 3D CNN-LSTM model for spatiotemporal tasks (e.g., video classification).

    Parameters:
    - input_shape: Tuple of the shape (num_frames, height, width, channels), e.g., (16, 128, 128, 3)
    - num_classes: The number of output classes for classification.

    Returns:
    - model: A compiled Keras model.
    """

    # Define input layer using Input()
    inputs = Input(shape=input_shape)

    # 3D CNN layers for spatial and temporal feature extraction
    x = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', padding='same')(inputs) # Pass input to the first layer
    x = MaxPooling3D(pool_size=(1, 2, 2), padding='same')(x)  # Pool only over height and width, not depth

    x = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), padding='same')(x)

    x = Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), padding='same')(x)

    # Flatten the 3D CNN output and reshape for the LSTM
    x = Flatten()(x)
    x = Reshape((20, -1))(x) # Reshape to (timesteps, features) for LSTM

    # LSTM layers for temporal modeling
    x = Bidirectional(LSTM(128, return_sequences=True))(x)  # You can set return_sequences=True if stacking LSTMs

    # Dropout for regularization
    x = Dropout(0.5)(x)

    # Fully connected layer
    x = TimeDistributed(Dense(128, activation='relu'))(x)

    # Output layer with softmax for multi-class classification
    outputs = TimeDistributed(Dense(num_classes, activation='softmax'))(x)

    # Create the model using Model()
    model = Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [43]:
# prompt: fit model2

from tensorflow.keras.layers import Bidirectional as BiLSTM

model2 = create_3dcnnlstm(input_shape, num_classes)
model2.summary()

# Fit the model
model2.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 5, 128, 128, 3)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_3 (Conv3D)                    │ (None, 5, 128, 128, 32)     │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_3 (MaxPooling3D)       │ (None, 5, 64, 64, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_4 (Conv3D)                    │ (None, 5, 64, 64, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_4 (MaxPooling3D)       │ (None, 5, 32, 32, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_5 (Conv3D)                    │ (None, 5, 32, 32, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_5 (MaxPooling3D)       │ (None, 5, 16, 16, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 163840)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 20, 8192)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 20, 256)             │       8,520,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 20, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 20, 128)             │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 20, 1682)            │         216,978 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,049,874 (34.52 MB)

 Trainable params: 9,049,874 (34.52 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 11s 331ms/step - accuracy: 0.3682 - loss: 5.5547 - val_accuracy: 0.4625 - val_loss: 3.7443
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 272ms/step - accuracy: 0.4329 - loss: 3.8522 - val_accuracy: 0.4625 - val_loss: 3.6349
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 265ms/step - accuracy: 0.4325 - loss: 3.7651 - val_accuracy: 0.4625 - val_loss: 3.5840
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 260ms/step - accuracy: 0.4361 - loss: 3.6193 - val_accuracy: 0.4625 - val_loss: 3.4685
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 270ms/step - accuracy: 0.4573 - loss: 3.5080 - val_accuracy: 0.5200 - val_loss: 3.4010
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 267ms/step - accuracy: 0.5003 - loss: 3.3358 - val_accuracy: 0.5222 - val_loss: 3.3615
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 272ms/step - accuracy: 0.5010 - loss: 3.3521 - val_accuracy: 0.5222 - val_loss: 3.3615
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 10s 263ms/step - accuracy: 0.4953 - loss: 3.3575 - val_accur

In [46]:
y_data.shape

(1000, 20)

In [37]:
!pip install rouge-score # Install the rouge-score package using pip


In [47]:
import pickle as pkl
pkl.dump(model, open('model2.pkl', 'wb'))
# Load the saved model
loaded_model = pkl.load(open('model2.pkl', 'rb'))



In [48]:
# prompt: download model

from google.colab import files

files.download('model2.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
# prompt: use loaded_model to generate captions for test.gif

import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import imageio
import numpy as np
from google.colab import drive
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, LSTM, TimeDistributed, Reshape
from nltk.translate.bleu_score import corpus_bleu

import pickle as pkl
import os


# Assuming you have 'loaded_model' and 'tokenizer' from previous steps

def generate_caption_for_gif(gif_path, model, tokenizer, max_caption_length=20):
    """
    Generates a caption for a GIF using a trained model.

    Args:
      gif_path: The path to the GIF file.
      model: The trained model for caption generation.
      tokenizer: The tokenizer used during training.
      max_caption_length: The maximum length of the generated caption.

    Returns:
      The generated caption.
    """

    # Step 1: Extract frames from the GIF and convert to numpy array
    frames = extract_and_save_frames(gif_path)
    video_array = video_to_array(frames)
    video_array = np.expand_dims(video_array, axis=0)

    # Step 2: Use the model to predict the caption
    predicted_sequence = model.predict(video_array)[0]


    # Step 3: Convert the predicted sequence back to text
    predicted_sequence = np.argmax(predicted_sequence, axis=-1)
    print(predicted_sequence)
    predicted_caption = tokenizer.sequences_to_texts([predicted_sequence])[0]


    return predicted_caption

# Example usage
test_gif_path = '/content/drive/My Drive/test2.gif'  # Replace with the actual path to your test GIF
if os.path.exists(test_gif_path):
  predicted_caption = generate_caption_for_gif(test_gif_path, loaded_model, tokenizer)
  print(f"Predicted Caption for test.gif: {predicted_caption}")
else:
  print("test2.gif not found")



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step
[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Predicted Caption for test.gif: a a a a a a a a
